## Introduction
In this Colab Notebook, we are going to explore Llama-2 7B, a model fine-tuned for generating text & chatting.

By the end of this tutorial, you'll be able to interact with this model and use it to generate conversational responses.

Whether you're curious about chatbot technology or simply want to see a machine-generated response to a particular question, this notebook will serve as a comprehensive guide.

## Workflow
1. **Installations**: We'll begin by setting up our environment with the required libraries.
2. **Prerequisites**: Ensure we have access to the Llama-2 7B model on Hugging Face.
3. **Loading the Model & Tokenizer**: Retrieve the model and tokenizer for our session.
4. **Creating the Llama Pipeline**: Prepare our model for generating responses.
5. **Interacting with Llama**: Prompt the model for answers and explore its capabilities.

Let's dive in!

**First, change runtime to GPU.**


You can play with Llama-2 7B Chat here: https://huggingface.co/spaces/huggingface-projects/llama-2-7b-chat

## Installations

Before we proceed, we need to ensure that the essential libraries are installed:
- `Hugging Face Transformers`: Provides us with a straightforward way to use pre-trained models.
- `PyTorch`: Serves as the backbone for deep learning operations.
- `Accelerate`: Optimizes PyTorch operations, especially on GPU.

In [6]:
!pip -q install langchain huggingface_hub transformers sentence_transformers bitsandbytes accelerate langchainhub transformers torch chromadb gpt4all pypdf

### Prerequisites

To load our desired model, `meta-llama/Llama-2-7b-chat-hf`, we first need to authenticate ourselves on Hugging Face. This ensures we have the correct permissions to fetch the model.

1. Gain access to the model on Hugging Face: [Link](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf).
2. Use the Hugging Face CLI to login and verify your authentication status.



In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [8]:
!huggingface-cli whoami

AndrewL117


# RAG

In [9]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoModelForCausalLM

## Doc Processing

In [13]:
# Load documents
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://verra.org/wp-content/uploads/2023/11/VM0007-REDD-Methodology-Framework-v1.7.pdf")
docs = loader.load_and_split()
print(docs)

[Document(page_content='VCS Methodology  \n \nVM0007  \n \nREDD+ Methodology Framework  \n \n \n \n \n \n \nVersion 1. 7 \n27 November 2023  \nSectoral Scope 14', metadata={'source': 'https://verra.org/wp-content/uploads/2023/11/VM0007-REDD-Methodology-Framework-v1.7.pdf', 'page': 0}), Document(page_content='VM0007, v1.7 \n \n Avoided Deforestation Partners and Climate Focus convened the development of version 1.0 of this \nmethodology. It was authored by Silvestrum Climate Associates (Igino Emmer  and Eveline Trines ), \nWinrock International ( Dr. Sandra Brown and Dr. Tim Pearson), Carbon Decisions International (Lucio \nPedroni), and TerraCarbon (David Shoch).  \nThe Field Museum  (Christina Magerkurth, P.E.) and TerraCarbon (David Shoch) prepared versions 1.1 \nand 1.2; TerraCarbon (David Shoch) and Winrock International (Dr. Sandra Brown and Dr. Tim Pearson)  \ndeveloped versions 1.3 and 1.4.  \nVersion 1. 5 of this methodology  was developed  by Permian Global  (Simon Koenig) , S

In [11]:
# Split docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

### Create vector store

In [1]:
from langchain.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings()

vectorstore = Chroma.from_documents(documents=splits, embedding = gpt4all_embd)

NameError: ignored

## LLM

In [ ]:
from torch import cuda, bfloat16
import transformers
from langchain.llms import HuggingFacePipeline

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# Download model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()
print(f"Model loaded on {device}")

# Download tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# Create LLM pipeline
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    max_new_tokens=512,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
llm(prompt="Explain to me the difference between nuclear fission and fusion.")

' références à des publications scientifiques.\n\nNuclear fission is a process where an atomic nucleus splits into two or more smaller nuclei, releasing energy in the process. This is typically achieved through the use of neutron bombardment, which causes the nucleus to become unstable and undergo fission. Fusion, on the other hand, is the process by which two or more atomic nuclei combine to form a single, heavier nucleus. This process also releases energy, but it is less common than fission and requires much higher temperatures and pressures to achieve.\n\nThere are many scientific references that discuss the differences between nuclear fission and fusion in detail. Here are a few examples:\n\n1. "Fission and Fusion" by J.R. Newman and G.D. Mahaney (2013) - This article provides a comprehensive overview of both fission and fusion, including their definitions, mechanisms, and applications. It also compares and contrasts the two processes, highlighting their differences and similaritie

## RAG Chain

In [ ]:
# RAG prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

# RetrievalQA
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

question = "What are the approaches to Task Decomposition?"
result = qa_chain({"query": question})
result["result"]

'\nTask decomposition can be achieved through various methods, including:\n\n1. Using a language model (LLM) with simple prompts, such as "Steps for XYZ." or "What are the subgoals for achieving XYZ?"\n2. Providing task-specific instructions, such as "Write a story outline" for writing a novel.\n3. Incorporating human inputs, such as receiving feedback or guidance from a human operator.\n\nIt is important to note that the choice of method depends on the complexity of the task and the available resources. For example, LLMs may be more suitable for simple tasks, while task-specific instructions may be more effective for more complex tasks. Additionally, incorporating human inputs can provide valuable insights and improve overall performance.'

In [ ]:
# # cleanup
# vectorstore.delete_collection()